In [2]:
pip install keras

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
from matplotlib import pyplot as plt
# %matplotlib inline
import numpy as np
import pandas as pd
import h5py
import os.path
import seaborn as sns
from scipy import stats

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GRU
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, GlobalAveragePooling1D
from keras.utils import np_utils

In [5]:
# 센서 1만 데이터 불러오기
# h5 데이터는 이 파일과 같은 폴더에 있어야 함... 죄송함댜ㅎㅎ
Label_size = 32
sensor1_data = np.empty((0,765+Label_size),int)
position_label = np.empty((0,Label_size),int)
label_cnt = 0
sensor1_data_training = np.empty((0,765+Label_size),int)
sensor1_data_test = np.empty((0,765+Label_size),int)
print(position_label)
for ii in range(12346):
#for ii in range(2):
    filename ='20201022_Depth_{}.h5'.format(ii)
    
    if os.path.isfile(filename):
        h5f = h5py.File(filename,'r')
        a_dset_keys = list(h5f.keys())
        ds_data = h5f[a_dset_keys[0]]
        sensor1 = ds_data[:,0,:]    
        
        sensor1 = np.array(sensor1)
        position_label = np.zeros((sensor1.shape[0],Label_size))
        #print(position_label.shape)
        #32bit coding
        if(ii == 0):
            aa = 1
        else:
            position_label[:,label_cnt] = 1
        label_cnt += 1
        
        sensor1 = np.append(sensor1, position_label, axis=1)
        
        sensor_len = len(sensor1)
        columns = range(0,sensor_len)
        training_per = round(sensor_len*0.8)
        test_per = sensor_len-training_per
        training_data = sensor1[:training_per]
        test_data = sensor1[training_per+1:]
        
        sensor1_data_training = np.append(sensor1_data_training, training_data, axis = 0)
        sensor1_data_test = np.append(sensor1_data_test,test_data,axis = 0)
        print("training_data:", sensor1_data_training.shape)
        print("test_data",sensor1_data_test.shape)
        
        sensor1_data= np.append(sensor1_data,sensor1,axis = 0)
        
        #print(sensor1_data.shape)
        #print(sensor1_data)
        
        
        #sensor1_data_training = np.append(sensor1_data_training, sensor1[:training_per], axis = 0)
        #sensor1_data_test= np.append(sensor1_data_test, sensor1[training_per+1:], axis = 0)
        #sensor1_data_training = np.append(sensor1_data_training, sensor1_data_training_current,axis = 0)
        #sensor1_data_test = np.append(sensor1_data_test,sensor1_data_test_current,axis = 0)
        #print("Total data length:",sensor1_data.shape)
        #print("training data length:",sensor1_data_training.shape)
        #print("test data length:",sensor1_data_test.shape)
        
        print("{} done".format(ii))
    else:
        a = 1
        

position_label.shape

[]
training_data: (280, 797)
test_data (69, 797)
0 done
training_data: (2933, 797)
test_data (731, 797)
1 done
training_data: (5387, 797)
test_data (1344, 797)
2 done
training_data: (7705, 797)
test_data (1923, 797)
3 done
training_data: (9984, 797)
test_data (2492, 797)
4 done
training_data: (13285, 797)
test_data (3316, 797)
5 done
training_data: (14575, 797)
test_data (3637, 797)
12 done
training_data: (17022, 797)
test_data (4248, 797)
13 done
training_data: (19363, 797)
test_data (4832, 797)
14 done
training_data: (20048, 797)
test_data (5002, 797)
15 done
training_data: (22045, 797)
test_data (5500, 797)
23 done
training_data: (24070, 797)
test_data (6005, 797)
24 done
training_data: (25755, 797)
test_data (6425, 797)
25 done
training_data: (27073, 797)
test_data (6754, 797)
34 done
training_data: (28070, 797)
test_data (7002, 797)
35 done
training_data: (28996, 797)
test_data (7233, 797)
45 done
training_data: (31245, 797)
test_data (7794, 797)
123 done
training_data: (33313, 79

(2179, 32)

In [11]:
sc = preprocessing.StandardScaler()
#Training Data, test Data => DataFrame화
sensor_len_train = len(sensor1_data_training)
columns_train = range(0,sensor_len_train)
sensor_len_test = len(sensor1_data_test)
columns_test = range(0,sensor_len_test)

df_train = pd.DataFrame(data = sensor1_data_training, index = columns_train)
df_test = pd.DataFrame(data = sensor1_data_test, index = columns_test)

#Scaling
#df_train[:,:764] = sc.fit_transform(df_train[:764])
#df_test[:,:764] = sc.transform(df_test[:,:764])

# train 데이터,Test 데이터 준비
X_train = df_train.loc[:,:764]
X_test = df_test.loc[:,:764]
#y_train = df_train.loc[:,765:]
#y_test = df_test.loc[:,765:]
y_train = sensor1_data_training[:,765:]
y_test = sensor1_data_test[:,765:]

#Scaling
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#CNN 모델 적용을 위한 reshape
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))
#X_train = X_train.values.reshape((X_train.shape[0],X_train.shape[1],1))
#X_test = X_test.values.reshape((X_test.shape[0],X_test.shape[1],1))
#y_train.shape
#y_train
y_test
X_test

array([[[-1.1789647 ],
        [-1.17934984],
        [-1.18168469],
        ...,
        [ 0.57553962],
        [ 0.57554661],
        [ 0.57733117]],

       [[-1.17466774],
        [-1.17514132],
        [-1.17756085],
        ...,
        [ 0.49354402],
        [ 0.49355093],
        [ 0.49531416]],

       [[-1.19615253],
        [-1.19618389],
        [-1.19611813],
        ...,
        [ 0.74773039],
        [ 0.74773755],
        [ 0.7495669 ]],

       ...,

       [[ 0.63864834],
        [ 0.64083185],
        [ 0.6410522 ],
        ...,
        [-0.12962255],
        [-0.12961626],
        [-0.13621684]],

       [[ 0.75466619],
        [ 0.75446169],
        [ 0.75445778],
        ...,
        [-0.23621683],
        [-0.23621065],
        [-0.24283895]],

       [[ 0.61716355],
        [ 0.61558078],
        [ 0.61630917],
        ...,
        [-0.31001287],
        [-0.31000677],
        [-0.31665426]]])

In [ ]:
#add_feature를 위한 함수 정의
# 우리 데이터 형태에 맞게 수정-> 현재 Column 인덱스를 숫자 0~764로 정의
def find_peak_global_rolling(df, thres, window):
    df_peak = df[np.abs(df) > thres]
    df_peak.columns = df.columns + '_peak'

    df_count = df_peak.rolling(window, min_periods=1).count()
    df_count.columns = df.columns + '_count'+ str(window)
    return df_peak, df_count
def find_peak_global(df, thres):
    df_peak = df[np.abs(df) > thres]
    df_peak.columns = df.columns + '_peak'
    return df_peak
def rolling_max(df, window):
    df_max = df.rolling(window, min_periods=1).max()
    #df_max.columns = [col + "_max" + str(window) for col in df.columns]
    df_max.columns = [col for col in df.columns]
    return df_max
def rolling_std(df, window):
    df_std = df.rolling(window, min_periods=1).std().fillna(0)
    #df_std.columns = [col + "_std" + str(window) for col in df.columns]
    df_std.columns = [col for col in df.columns]
    return df_std
def rolling_average(df, window):
    df_av = df.rolling(window, min_periods=1).mean()
    #df_av.columns = [col + "_av" + str(window) for col in df.columns]
    df_av.columns = [col for col in df.columns]
    return df_av

In [12]:
#CNN모델
model_m = Sequential()
model_m.add(Conv1D(64, 20, activation='relu',input_shape=(765,1)))
model_m.add(Conv1D(64, 20, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(128, 20, activation='relu'))
model_m.add(GlobalAveragePooling1D())
#model_m.add(MaxPooling1D(3))
#model_m.add(Flatten())
model_m.add(Dropout(0.2))
model_m.add(Dense(32, activation='softmax'))
print(model_m.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 746, 64)           1344      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 727, 64)           81984     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 242, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 223, 128)          163968    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4

In [ ]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
]
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

history = model_m.fit(X_train,
                      y_train,
                      batch_size=50,
                      epochs=2,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

Epoch 1/2
941/941 [==============================] - 561s 596ms/step - loss: 0.6964 - accuracy: 0.7684 - val_loss: 29.8395 - val_accuracy: 0.1634
Epoch 2/2
268/941 [=======>......................] - ETA: 6:01 - loss: 0.0572 - accuracy: 0.9774

In [ ]:
def show_results(y_test, y_pred):
    max_y_pred_test = np.argmax(y_pred, axis=1)
    max_y_test = np.argmax(y_test, axis=1)
    show_confusion_matrix(max_y_test, max_y_pred_test)
    print(classification_report(max_y_test, max_y_pred_test))
    

In [ ]:
y_test.shape

In [ ]:
np.argmax(y_test, axis =1)

In [ ]:
y_pred.shape

In [ ]:
y_pred = model_m.predict(X_test)
show_results(y_test, y_pred)